### Import py_snap_helpers functions and classes

In [1]:
from py_snap_helpers import *


### Get the human readable information about a SNAP operator

In [2]:
operator = 'Calibration'

op_help(operator)

Operator name: org.esa.s1tbx.calibration.gpf.CalibrationOp
Operator alias: Calibration

Parameters:

sourceBandNames: The list of source bands.
Default Value: None

Possible values: []

auxFile: The auxiliary file
Default Value: Latest Auxiliary File

Possible values: ['Latest Auxiliary File', 'Product Auxiliary File', 'External Auxiliary File']

externalAuxFile: The antenna elevation pattern gain auxiliary data file.
Default Value: None

Possible values: []

outputImageInComplex: Output image in complex
Default Value: false

Possible values: []

outputImageScaleInDb: Output image scale
Default Value: false

Possible values: []

createGammaBand: Create gamma0 virtual band
Default Value: false

Possible values: []

createBetaBand: Create beta0 virtual band
Default Value: false

Possible values: []

selectedPolarisations: The list of polarisations
Default Value: None

Possible values: []

outputSigmaBand: Output sigma0 band
Default Value: true

Possible values: []

outputGammaBand: Outpu

### Get the operator parameter default values

In [3]:
parameters = get_operator_default_parameters(operator)

In [4]:
parameters

{'auxFile': 'Latest Auxiliary File',
 'createBetaBand': 'false',
 'createGammaBand': 'false',
 'externalAuxFile': None,
 'outputBetaBand': 'false',
 'outputGammaBand': 'false',
 'outputImageInComplex': 'false',
 'outputImageScaleInDb': 'false',
 'outputSigmaBand': 'true',
 'selectedPolarisations': None,
 'sourceBandNames': None}

Update the values:

In [5]:
parameters['outputBetaBand'] = 'true'

In [6]:
parameters

{'auxFile': 'Latest Auxiliary File',
 'createBetaBand': 'false',
 'createGammaBand': 'false',
 'externalAuxFile': None,
 'outputBetaBand': 'true',
 'outputGammaBand': 'false',
 'outputImageInComplex': 'false',
 'outputImageScaleInDb': 'false',
 'outputSigmaBand': 'true',
 'selectedPolarisations': None,
 'sourceBandNames': None}

### Create a simple SNAP Graph

Instantiate a Graph:

In [7]:
mygraph = GraphProcessor()

In [8]:
op_help('Read')

Operator name: org.esa.snap.core.gpf.common.ReadOp
Operator alias: Read

Parameters:

file: The file from which the data product is read.
Default Value: None

Possible values: []

formatName: An (optional) format name.
Default Value: None

Possible values: []



Let's create the first node of the Graph where:

- the variable operator is the name of the SNAP Operator
- the variable node_id is the name of the node being created
- the variable source_node_id is the source node(s) - Read is the first node so there isn't any source node

In [9]:
operator = 'Read'

node_id = 'Read'

source_node_id = ''

In [10]:
parameters = get_operator_default_parameters(operator)

In [11]:
parameters

{'file': None, 'formatName': None}

Replace the default value for the parameter file with the path to the Sentinel-1 product

In [12]:
parameters['file'] = '/workspace/data/S1B_EW_GRDM_1SDH_20180822T114138_20180822T114238_012375_016D07_E770.zip'

Add the Read node to the Graph:

In [13]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

At this stage, the Graph looks like:

In [14]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1B_EW_GRDM_1SDH_20180822T114138_20180822T114238_012375_016D07_E770.zip</file>
    </parameters>
  </node>
</graph>



**ThermalNoiseRemoval**

Thermal noise correction can be applied to Sentinel-1 Level-1 SLC products as well as Level-1 GRD products which have not already been corrected. The operator can also remove this correction based on the product annotations (i.e. to re-introduce the noise signal that was removed). Product annotations will be updated accordingly to allow for re-application of the correction.

Level-1 products provide a noise LUT for each measurement data set. The values in the de-noise LUT, provided in linear power, can be used to derive calibrated noise profiles matching the calibrated GRD data.   

Bi-linear interpolation is used for any pixels that fall between points in the LUT.

Now for the ThermalNoiseRemoval Operator we follow a similar process:

In [15]:
op_help('ThermalNoiseRemoval')

Operator name: org.esa.s1tbx.calibration.gpf.Sentinel1RemoveThermalNoiseOp
Operator alias: ThermalNoiseRemoval

Parameters:

selectedPolarisations: The list of polarisations
Default Value: None

Possible values: []

removeThermalNoise: Remove thermal noise
Default Value: true

Possible values: []

reIntroduceThermalNoise: Re-introduce thermal noise
Default Value: false

Possible values: []



In [16]:
operator = 'ThermalNoiseRemoval'

node_id = 'ThermalNoiseRemoval' 

source_node_id = 'Read'

parameters = get_operator_default_parameters(operator)

mygraph.add_node(node_id, operator, parameters, source_node_id)

**Apply-Orbit-File**

The orbit state vectors provided in the metadata of a SAR product are generally not accurate and can be refined with the precise orbit files which are available days-to-weeks after the generation of the product.  The orbit file provides accurate satellite position and velocity information. Based on this information, the orbit state vectors in the abstract metadata of the product are updated.

For Sentinel-1, Restituted orbit files and Precise orbit files may be applied. Precise orbits are produced a few weeks after acquisition. Orbit files are automatically download from Array's servers by SNAP. If an orbit file is not found, you may try looking for it in https://qc.sentinel1.eo.esa.int/  and placing the downloaded file into the auxdata folder.

In [17]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File' 

source_node_id = 'ThermalNoiseRemoval'

parameters = get_operator_default_parameters(operator)

parameters

{'continueOnFail': 'false',
 'orbitType': 'Sentinel Precise (Auto Download)',
 'polyDegree': '3'}

In [18]:
mygraph.add_node(node_id, operator, parameters, source_node_id)


**Calibration**

The objective of SAR calibration is to provide imagery in which the pixel values can be directly related to the radar backscatter of the scene. Though uncalibrated SAR imagery is sufficient for qualitative use, calibrated SAR images are essential to quantitative use of SAR data.

Typical SAR data processing, which produces level 1 images, does not include radiometric corrections and significant radiometric bias remains. Therefore, it is necessary to apply the radiometric correction to SAR images so that the pixel values of the SAR images truly represent the radar backscatter of the reflecting surface. The radiometric correction is also necessary for the comparison of SAR images acquired with different sensors, or acquired from the same sensor but at different times, in different modes, or processed by different processors.

This Operator performs different calibrations for ASAR, ERS, ALOS and Radarsat-2 products deriving the sigma nought images. Optionally gamma nought and beta nought images can also be created.

In [19]:
operator = 'Calibration'

node_id = 'Calibration' 

source_node_id = 'Apply-Orbit-File'

parameters = get_operator_default_parameters(operator)

parameters

{'auxFile': 'Latest Auxiliary File',
 'createBetaBand': 'false',
 'createGammaBand': 'false',
 'externalAuxFile': None,
 'outputBetaBand': 'false',
 'outputGammaBand': 'false',
 'outputImageInComplex': 'false',
 'outputImageScaleInDb': 'false',
 'outputSigmaBand': 'true',
 'selectedPolarisations': None,
 'sourceBandNames': None}

In [20]:
mygraph.add_node(node_id, operator, parameters, source_node_id)


**Speckle-Filter**

SAR images have inherent salt and pepper like texturing called speckles which degrade the quality of the image and make interpretation of features more difficult. Speckles are caused by random constructive and destructive interference of the de-phased but coherent return waves scattered by the elementary scatters within each resolution cell. Speckle noise reduction can be applied either by spatial filtering or multilook processing.

In [21]:
operator = 'Speckle-Filter'

node_id = 'Speckle-Filter' 

source_node_id = 'Calibration'

parameters = get_operator_default_parameters(operator)

parameters

{'anSize': '50',
 'dampingFactor': '2',
 'enl': '1.0',
 'estimateENL': 'false',
 'filter': 'Lee Sigma',
 'filterSizeX': '3',
 'filterSizeY': '3',
 'numLooksStr': '1',
 'sigmaStr': '0.9',
 'sourceBandNames': None,
 'targetWindowSizeStr': '3x3',
 'windowSize': '7x7'}

In [22]:
mygraph.add_node(node_id, operator, parameters, source_node_id)


**Multilook**

Generally, a SAR original image appears speckled with inherent speckle noise. To reduce this inherent speckled appearance, several images are incoherently combined as if they corresponded to different looks of the same scene. This processing is generally known as multilook processing. As a result the multilooked image improves the image interpretability. Additionally, multilook processing can be used to produce an application product with nominal image pixel size.

_Multilook Method_

There are two ways to implement the multilook processing:

- The multilooked images can be produced by space-domain averaging of a single look image, either with or without specific 2D kernels by convolution.
- The multilook images can be produced by frequency-domain method using the sub-spectral band width.    This operator implements the space-domain multilook method by averaging a single look image with a small sliding window.

_Selecting Range and Azimuth Looks_

In selecting the number of range looks and the number of azimuth looks, user has two options:

- GR square pixel: the user specifies the number of range looks while the number of azimuth looks is computed based on the ground range spacing and the azimuth spacing. The window size is then determined by the number of range looks and the number of azimuth looks. As a result, image with approximately square pixel spacing on the ground is produced.
- Independent looks: the number of looks in range and azimuth can be selected independently. The window size is then determined by the number of range looks and the number of azimuth looks.

In [23]:
operator = 'Multilook'

node_id = 'Multilook' 

source_node_id = 'Speckle-Filter'

parameters = get_operator_default_parameters(operator)

parameters

{'grSquarePixel': 'true',
 'nAzLooks': '1',
 'nRgLooks': '1',
 'outputIntensity': 'false',
 'sourceBandNames': None}

In [24]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

**LinearToFromdB**

Converts bands to dB

In [25]:
operator = 'LinearToFromdB'

node_id = 'LinearToFromdB' 

source_node_id = 'Multilook'

parameters = get_operator_default_parameters(operator)

parameters

{'sourceBandNames': None}

In [26]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

**Terrain correction**


In [27]:
operator = 'Terrain-Correction'

map_proj = """PROJCS["WGS 84 / Arctic Polar Stereographic", 
  GEOGCS["WGS 84", 
    DATUM["World Geodetic System 1984", 
      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
      AUTHORITY["EPSG","6326"]], 
    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
    UNIT["degree", 0.017453292519943295], 
    AXIS["Geodetic longitude", EAST], 
    AXIS["Geodetic latitude", NORTH], 
    AUTHORITY["EPSG","4326"]], 
  PROJECTION["Polar Stereographic (variant B)", AUTHORITY["EPSG","9829"]], 
  PARAMETER["central_meridian", 0.0], 
  PARAMETER["Standard_Parallel_1", 71.0], 
  PARAMETER["false_easting", 0.0], 
  PARAMETER["false_northing", 0.0], 
  UNIT["m", 1.0], 
  AXIS["Easting", "South along 90 deg East"], 
  AXIS["Northing", "South along 180 deg"], 
  AUTHORITY["EPSG","3995"]]"""

node_id = 'Terrain-Correction' 

source_node_id = 'LinearToFromdB'

parameters = get_operator_default_parameters(operator)

parameters['demName'] = 'ACE30'  
parameters['saveDEM'] = 'true'
parameters['mapProjection'] = map_proj
parameters['nodataValueAtSea'] = 'false'   
                
parameters

{'alignToStandardGrid': 'false',
 'applyRadiometricNormalization': 'false',
 'auxFile': 'Latest Auxiliary File',
 'demName': 'ACE30',
 'demResamplingMethod': 'BILINEAR_INTERPOLATION',
 'externalAuxFile': None,
 'externalDEMApplyEGM': 'true',
 'externalDEMFile': None,
 'externalDEMNoDataValue': '0',
 'imgResamplingMethod': 'BILINEAR_INTERPOLATION',
 'incidenceAngleForGamma0': 'Use projected local incidence angle from DEM',
 'incidenceAngleForSigma0': 'Use projected local incidence angle from DEM',
 'mapProjection': 'PROJCS["WGS 84 / Arctic Polar Stereographic", \n  GEOGCS["WGS 84", \n    DATUM["World Geodetic System 1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Geodetic longitude", EAST], \n    AXIS["Geodetic latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Polar Stereographic (var

In [28]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

**Subset**

In [29]:
operator = 'Subset'

node_id = 'Subset' 

source_node_id = 'Terrain-Correction'

parameters = get_operator_default_parameters(operator)

parameters['geoRegion'] = 'POLYGON ((-35.2717796499525 83.2658579085283, -29.3687708669507 83.8704727024008, -35.334759858866 84.53393836008161, -41.2248291287458 83.8638684209569, -35.2717796499525 83.2658579085283))'

parameters

{'bandNames': None,
 'copyMetadata': 'false',
 'fullSwath': 'false',
 'geoRegion': 'POLYGON ((-35.2717796499525 83.2658579085283, -29.3687708669507 83.8704727024008, -35.334759858866 84.53393836008161, -41.2248291287458 83.8638684209569, -35.2717796499525 83.2658579085283))',
 'region': None,
 'subSamplingX': '1',
 'subSamplingY': '1',
 'tiePointGridNames': None}

In [30]:
mygraph.add_node(node_id, operator, parameters, source_node_id)


**Write**

Add the Write node:

In [31]:
op_help('Write')

Operator name: org.esa.snap.core.gpf.common.WriteOp
Operator alias: Write

Parameters:

file: The output file to which the data product is written.
Default Value: None

Possible values: []

formatName: The name of the output file format.
Default Value: BEAM-DIMAP

Possible values: []

deleteOutputOnFailure: If true, all output files are deleted after a failed write operation.
Default Value: true

Possible values: []

writeEntireTileRows: If true, the write operation waits until an entire tile row is computed.
Default Value: true

Possible values: []

clearCacheAfterRowWrite: If true, the internal tile cache is cleared after a tile row has been written. Ignored if writeEntireTileRows=false.
Default Value: false

Possible values: []



In [32]:
get_write_formats()

PolSARPro (.hdr)
GDAL-RMF-WRITER (.rsw)
GDAL-GS7BG-WRITER (.grd)
GDAL-HFA-WRITER (.img)
Snaphu (.hdr)
NetCDF-BEAM (.nc)
HDF5 (.h5)
ENVI (.hdr)
NetCDF4-BEAM (.nc)
GDAL-KRO-WRITER (.kro)
GDAL-PNM-WRITER (.pnm)
GeoTIFF (.tif)
GDAL-GTiff-WRITER (.tif)
NetCDF-CF (.nc)
GDAL-SGI-WRITER (.rgb)
GeoTIFF+XML (.tif)
GDAL-GSBG-WRITER (.grd)
JP2 ()
BEAM-DIMAP (.dim)
GeoTIFF-BigTIFF (.tif)
GDAL-ILWIS-WRITER (.mpr)
GDAL-BMP-WRITER (.bmp)
GDAL-BT-WRITER (.bt)
CSV (.csv)
JPEG2000 (.jp2)
Gamma (.par)
GDAL-MFF-WRITER (.hdr)
NetCDF4-CF (.nc)
GDAL-RST-WRITER (.rst)
GDAL-GTX-WRITER (.gtx)
GDAL-NITF-WRITER (.ntf)
GDAL-PCIDSK-WRITER (.pix)
GDAL-SAGA-WRITER (.sdat)
Generic Binary BSQ (.bin)


In [33]:
operator = 'Write'

node_id = 'Write'

source_node_id = 'Subset'

parameters = get_operator_default_parameters(operator)

parameters

{'clearCacheAfterRowWrite': 'false',
 'deleteOutputOnFailure': 'true',
 'file': None,
 'formatName': 'BEAM-DIMAP',
 'writeEntireTileRows': 'true'}

In [34]:
parameters['file'] = 'sigma0'
parameters['formatName'] = 'GeoTIFF-BigTiff'

In [35]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

View graph:

In [36]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1B_EW_GRDM_1SDH_20180822T114138_20180822T114238_012375_016D07_E770.zip</file>
    </parameters>
  </node>
  <node id="ThermalNoiseRemoval">
    <operator>ThermalNoiseRemoval</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <reIntroduceThermalNoise>false</reIntroduceThermalNoise>
      <selectedPolarisations/>
      <removeThermalNoise>true</removeThermalNoise>
    </parameters>
  </node>
  <node id="Apply-Orbit-File">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="ThermalNoiseRemoval"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</o

Run the Graph with:

```python
mygraph.run()
```

### Example of linear SNAP graph

If the SNAP Graph does not include fan-in/out, i.e. it is linear, there's an optional approach using py_snap_helpers.

Below an example to pre-process Sentinel-3 SLSTR Level-1B data

In [ ]:
def pre_processing(product_metadata, **kwargs):
   
    options = dict()
    
    if product_metadata['orbitDirection'] == 'DESCENDING':
    
        operators = ['Read', 
                     'Rad2Refl',
                     'Resample',
                     'Reproject',
                     'Subset',
                     'AddLandCover',
                     'Write']
    else:
        
        operators = ['Read', 
                     'Resample',
                     'Reproject',
                     'Subset',
                     'AddLandCover',
                     'Write']
    
    for operator in operators:
            
        print 'Getting default values for Operator {}'.format(operator)
        parameters = get_operator_default_parameters(operator)
        
        options[operator] = parameters

    for key, value in kwargs.items():
        
        print 'Updating Operator {}'.format(key)
        options[key.replace('_', '-')].update(value)
    
    mygraph = GraphProcessor()
    
    for index, operator in enumerate(operators):
    
        print 'Adding Operator {} to graph'.format(operator)
        if index == 0:            
            source_node_id = ''
        
        else:
            source_node_id = operators[index - 1]
        
        mygraph.add_node(operator,
                         operator, 
                         options[operator], source_node_id)
    
    mygraph.run()


Next, create one Python dictionary per operator:

In [ ]:
read = dict()
read['file'] =  local_path
read['formatName'] = 'Sen3_SLSTRL1B_500m'

if s3_metadata['orbitDirection'] == 'DESCENDING':

    rad2refl = dict()
    rad2refl['sensor'] = 'SLSTR_500m'
    rad2refl['copyTiePointGrids'] = 'true'
    rad2refl['copyFlagBandsAndMasks'] = 'true'
    rad2refl['copyNonSpectralBands'] = 'true'

resample = dict()
resample['referenceBandName'] = 'F1_BT_in'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

subset = dict()
subset['geoRegion'] = aoi_wkt['value']

addlandcover = dict()
addlandcover['landCoverNames'] = 'CCILandCover-2015'

write = dict()
write['file'] = 'temp'

Finally, create and process the SNAP Graph with:

In [ ]:
if s3_metadata['orbitDirection'] == 'DESCENDING':

    pre_processing(product_metadata=s3_metadata, 
                   Read=read, 
                   Rad2Refl=rad2refl,
                   Resample=resample,
                   Reproject=reproject,
                   Subset=subset,
                   AddLandCover=addlandcover,
                   Write=write)
else:
    
    pre_processing(product_metadata=s3_metadata, 
                   Read=read, 
                   Resample=resample,
                   Reproject=reproject,
                   Subset=subset,
                   AddLandCover=addlandcover,
                   Write=write)